In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import load_model
from tensorflow import keras
from keras_preprocessing.sequence import TimeseriesGenerator

In [3]:
print(tf.test.is_built_with_cuda())

print(tf.test.is_gpu_available())

True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [4]:
AMDopen = pd.read_csv('BATS_MSFT_60.csv', delimiter=",", usecols=['open'],dtype={"open":object}, skip_blank_lines=0)
AMDclose = pd.read_csv('BATS_MSFT_60.csv', delimiter=",", usecols=['close'],dtype={"close":object}, skip_blank_lines=0)
AMDdate = pd.read_csv('BATS_MSFT_60.csv', delimiter=",", usecols=['time'],dtype={"time":object}, skip_blank_lines=0)

In [5]:
AMDopen = AMDopen.open.tolist()
AMDclose = AMDclose.close.tolist()
AMDdate = AMDdate.time.tolist()

AMDopen = [ float(x) for x in AMDopen if str(x) != 'nan']
AMDclose = [ float(x) for x in AMDclose if str(x) != 'nan']
AMDdate = [ x for x in AMDdate if x != 'nan']

print(len(AMDopen), len(AMDclose), len(AMDdate))

3320 3320 3320


In [6]:
AMDmid = []
for i in range(len(AMDopen)):
    AMDmid.append((AMDopen[i] + AMDclose[i])/2)

scaler=MinMaxScaler(feature_range=(0,1))
AMDmid=scaler.fit_transform(np.array(AMDmid).reshape(-1,1))

training_size = int(len(AMDmid)*.75)
test_size = len(AMDmid)-training_size
train_data,test_data=AMDmid[0:training_size],AMDmid[training_size:(len(AMDmid))]
#train_data = train_data.reshape(train_data.shape[0],train_data.shape[1] , 1)


generator_data = np.array(AMDmid)
print(generator_data.shape)

(3320, 1)


In [7]:
look_back = 2

generator_data = generator_data.reshape(len(generator_data),1)

print(generator_data.shape)

train_generator = tf.keras.preprocessing.sequence.TimeseriesGenerator(generator_data, generator_data, length=look_back, batch_size=5)
test_generator = TimeseriesGenerator(test_data, test_data, length=look_back, batch_size=1)

print("Samples %d" % len(train_generator))


(3320, 1)
Samples 664


In [8]:
model=Sequential()

model.add(LSTM(50, return_sequences=True, input_shape=(look_back,1)))
# model.add(LSTM(50, return_sequences=True))
# model.add(LSTM(50, return_sequences=True))
# model.add(LSTM(50, return_sequences=True))
# model.add(LSTM(100, return_sequences=True))
# model.add(Dropout(.2))
#model.add(LSTM(50, return_sequences=True))
#model.add(Dropout(0.2))


model.add(Flatten())
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam', metrics=['mean_squared_error'])

model.summary()

model.fit(train_generator, epochs=200, verbose=1)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 50)             10400     
_________________________________________________________________
flatten (Flatten)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 10,501
Trainable params: 10,501
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
